<a href="https://colab.research.google.com/github/jsvon/NLP-projects-/blob/main/Agent_for_text_to_SQL_with_automatic_error_correction5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Sql agent can inspect output and determine if outputs are to be changed or not.
# setting up SQL tables

from sqlalchemy import(
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    Float,
    insert,
    inspect,
    text
)

engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

# create city SQL table
table_name = "receipts"
receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("customer_name", String(16), primary_key=True),
    Column("price", Float),
    Column("tip", Float),
)
metadata_obj.create_all(engine)


In [2]:
rows = [
    {"receipt_id": 1, "customer_name": "Alan Payne", "price": 12.06, "tip": 1.20},
    {"receipt_id": 2, "customer_name": "Alex Mason", "price": 23.86, "tip": 0.24},
    {"receipt_id": 3, "customer_name": "Woodrow Wilson", "price": 53.43, "tip": 5.43},
    {"receipt_id": 4, "customer_name": "Margaret James", "price": 21.11, "tip": 1.00},
]

for row in rows:
  stmt = insert(receipts).values(**row)
  with engine.begin() as connection:
    cursor = connection.execute(stmt)

In [3]:
#checking the system works with basic query
with engine.connect() as con:
  rows = con.execute(text("SELECT * FROM receipts"))
  for row in rows:
    print(row)

(1, 'Alan Payne', 12.06, 1.2)
(2, 'Alex Mason', 23.86, 0.24)
(3, 'Woodrow Wilson', 53.43, 5.43)
(4, 'Margaret James', 21.11, 1.0)


In [4]:
#Building the agent

!pip install smolagents

from smolagents import tool

@tool
def sql_engine(query: str) -> str:
  """
  Allows you to perform SQL queries on the table. Returns a string representation of the result.
  The table is named 'receipts'.Its description is as follows.
     Columns:
     - receipt_id: INTEGER
     - customer_name: VARCHAR(16)
     - price: FLOAT
     - tip: FLOAT
  Args:
     query: The query to perform. This should be a valid SQL query.
  """

  output = ""
  with engine.connect() as con:
    rows = con.execute(text(query))
    for row in rows:
      output += str(row) + "\n"
  return output


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 85.1 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [5]:
from smolagents import CodeAgent, HfApiModel

agent = CodeAgent(
    tools = [sql_engine],
    model=HfApiModel(model_id='https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud')
)

agent.run("Can you give me the name of client who got the most expensive receipt")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me the name of client who got the most expensive receipt                                           │
│                                                                                                                 │
╰─ HfApiModel - https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud ───────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Query to get the customer name with the highest price                                                          
  query = "SELECT customer_name FROM receipts ORDER BY price DESC LIMIT 1;"                                        
  result = sql_engine(query=query)                                                                                 
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
('Woodrow Wilson',)


Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Woodrow Wilson")                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Woodrow Wilson

[Step 2: Duration 1.67 seconds| Input tokens: 4,469 | Output tokens: 148]

'Woodrow Wilson'

INCREASING DIFFICULTY: JOINS

In [9]:

table_name = "waiters"
# Check if the table already exists in metadata_obj
if table_name not in metadata_obj.tables:
  receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("waiter_name", String(16), primary_key=True),
)
metadata_obj.create_all(engine)
rows = [
    {"receipt_id": 1, "waiter_name": "Corey Johnson"},
    {"receipt_id": 2, "waiter_name": "Michael Watts"},
    {"receipt_id": 3, "waiter_name": "Michael Watts"},
    {"receipt_id": 4, "waiter_name": "Margaret James"},
]

# without error handling
for row in rows:
    stmt = insert(receipts).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)




IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: waiters.receipt_id, waiters.waiter_name
[SQL: INSERT INTO waiters (receipt_id, waiter_name) VALUES (?, ?)]
[parameters: (1, 'Corey Johnson')]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [10]:
# with error handling
for row in rows:
    stmt = insert(receipts).values(**row)
    try:
        with engine.begin() as connection:
            connection.execute(stmt)
    except Exception as e:
        print(f"Error inserting row {row}: {e}")

Error inserting row {'receipt_id': 1, 'waiter_name': 'Corey Johnson'}: (sqlite3.IntegrityError) UNIQUE constraint failed: waiters.receipt_id, waiters.waiter_name
[SQL: INSERT INTO waiters (receipt_id, waiter_name) VALUES (?, ?)]
[parameters: (1, 'Corey Johnson')]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
Error inserting row {'receipt_id': 2, 'waiter_name': 'Michael Watts'}: (sqlite3.IntegrityError) UNIQUE constraint failed: waiters.receipt_id, waiters.waiter_name
[SQL: INSERT INTO waiters (receipt_id, waiter_name) VALUES (?, ?)]
[parameters: (2, 'Michael Watts')]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
Error inserting row {'receipt_id': 3, 'waiter_name': 'Michael Watts'}: (sqlite3.IntegrityError) UNIQUE constraint failed: waiters.receipt_id, waiters.waiter_name
[SQL: INSERT INTO waiters (receipt_id, waiter_name) VALUES (?, ?)]
[parameters: (3, 'Michael Watts')]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
Error inserting row {'rec

In [7]:
updated_description = """Allows you to perform SQL queries on the table. Beware that this tool's output is a string representation of the execution output.
It can use the following tables:"""

inspector = inspect(engine)
for table in ["receipts", "waiters"]:
    columns_info = [(col["name"], col["type"]) for col in inspector.get_columns(table)]

    table_description = f"Table '{table}':\n"

    table_description += "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
    updated_description += "\n\n" + table_description

print(updated_description)

Allows you to perform SQL queries on the table. Beware that this tool's output is a string representation of the execution output.
It can use the following tables:

Table 'receipts':
Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT

Table 'waiters':
Columns:
  - receipt_id: INTEGER
  - waiter_name: VARCHAR(16)


In [8]:
sql_engine.description = updated_description

agent = CodeAgent(
    tools=[sql_engine],
    model=HfApiModel(model_id='https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud'),
)

agent.run("Which waiter got more total money from tips?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Which waiter got more total money from tips?                                                                    │
│                                                                                                                 │
╰─ HfApiModel - https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud ───────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Calculate total tips for each waiter                                                                           
  total_tips_query = """                                                                                           
  SELECT w.waiter_name, SUM(r.tip) AS total_tips                                                                   
  FROM waiters w                                                                                                   
  JOIN receipts r ON w.receipt_id = r.receipt_id                                                                   
  GROUP BY w.waiter_name                                                                                           
  ORDER BY total_tips DESC                                                                                         
  LIMIT 1;                                                                                                         
  """                                                                                                              
  print("Query to find the waiter with the most tips:", total_tips_query)                                          
  result = sql_engine(query=total_tips_query)                                                                      
  print("Result of the query:", result)                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Query to find the waiter with the most tips: 
SELECT w.waiter_name, SUM(r.tip) AS total_tips
FROM waiters w
JOIN receipts r ON w.receipt_id = r.receipt_id
GROUP BY w.waiter_name
ORDER BY total_tips DESC
LIMIT 1;

Result of the query: ('Michael Watts', 5.67)


Out: None

[Step 1: Duration 4.85 seconds| Input tokens: 2,155 | Output tokens: 166]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Michael Watts")                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Michael Watts

[Step 2: Duration 1.59 seconds| Input tokens: 4,730 | Output tokens: 217]

'Michael Watts'